In [1]:
# In The Name of God
# Train Adversary - 1
# Adversary Determines Age, using Dense121 with freezing FE.
# this model can use for extracing age from obfuscated data as an adversary
# the reverse of accuracy in this term means Privacy.
# changes:
#      V1: 25/12/1400 - base
#      V2: 26/12/1400 - model updated!
#      V3: 15/10/1401 - set gender as inferrable attribute

In [2]:
#@title
# Imports
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import numpy as np
from collections import OrderedDict
import time

from torch.utils.data import random_split
from math import floor

import torchvision.utils as vutils
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import matplotlib.animation as animation
from IPython.display import HTML
from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

import itertools
import random

import shutil
from zipfile import ZipFile
import os

from IPython.core.debugger import set_trace

In [3]:
# Hyper parameters:
# change lastRunEpochNumber for every run based on saved files
isFirstRun = False
lastRunEpochNumber = 'last'
manual_seed = 20
image_size = 64
use_whole_dataset = True
usage_percent = 1.0
celeba_male_index = 20
celeba_young_index = 39
celeba_smiling_index = 31
celeba_straight_hair_index = 32
celeba_wavy_hair_index = 33
celeba_black_hair_index = 8
celeba_blonde_hair_index = 9
celeba_brown_hair_index = 11
celeba_gray_hair_index = 17
celeba_eyeglasses_index = 15
celeba_chubby_index = 13
learning_rate = 0.001 #0.2
batch_size = 64

celeba_mouth_open_index = 21
celeba_high_cheekbone_index = 19

using_index = celeba_male_index

files_not_ready = True
dataset_folder_path = 'drive/MyDrive/ML/datasetShortcuts/CelebA'
#'C:/Workspace/Datasets/CelebA'
#'D:/on going/server/CelebA'
# dataset_folder_path = 'drive/MyDrive/ML/datasetShortcuts/CelebA'
data_dir = 'celeba'
save_dir = 'adv2-G'
saving_path = 'drive/MyDrive/ML/saves/' + save_dir + '/'
#'C:/Users/jamshidi/saveMLs/' + save_dir + '/'
#'C:/Users/Mohammad/savedML/' + save_dir + '/'
# saving_path = 'drive/MyDrive/ML/saves/' + save_dir + '/'

use_early_stop = False
use_rate_schedule = False

# Other params
# Number of workers for dataloader
workers = 2
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1
# Size of feature maps in encoder
nef = 64
# Size of feature maps in decoder
ndf = 64
# Number of channels in the training images. For color images this is 3
nc = 3
# Size of z latent vector (i.e. size of generator input)
nz = 100
# Number of training epochs
num_epochs = 200 #10

In [4]:
# Check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [5]:
# Mount google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# download dataset and unzip
# data_dir variable: data_dir = 'img_align_celeba'

if files_not_ready:
    dataset_zip_path = dataset_folder_path + '/Img/img_align_celeba.zip'
    list_eval_partition_path = dataset_folder_path + '/Eval/list_eval_partition.txt'
    identity_celeba_path = dataset_folder_path + '/Anno/identity_CelebA.txt'
    list_attr_celeba_path = dataset_folder_path + '/Anno/list_attr_celeba.txt'
    list_bbox_celeba_path = dataset_folder_path + '/Anno/list_bbox_celeba.txt'
    list_landmarks_align_celeba_path = dataset_folder_path + '/Anno/list_landmarks_align_celeba.txt'

    try:
      os.mkdir(data_dir)
      print("data folder created successfully")
    except OSError as e:
      print("Error: %s" % (e.strerror))

    shutil.copyfile(dataset_zip_path, data_dir + r'/img_align_celeba.zip')
    shutil.copyfile(list_eval_partition_path, data_dir + r'/list_eval_partition.txt')
    shutil.copyfile(identity_celeba_path, data_dir + r'/identity_CelebA.txt')
    shutil.copyfile(list_attr_celeba_path, data_dir + r'/list_attr_celeba.txt')
    shutil.copyfile(list_bbox_celeba_path, data_dir + r'/list_bbox_celeba.txt')
    shutil.copyfile(list_landmarks_align_celeba_path, data_dir + r'/list_landmarks_align_celeba.txt')

    try:
        shutil.rmtree(data_dir + r'/img_align_celeba')
        print("old unzipped directory removed successfully")
    except OSError as e:
        print("Error: %s" % (e.strerror))

    archive = data_dir + r'/img_align_celeba.zip'
    with ZipFile(archive, 'r') as zip:
       zip.extractall(data_dir)
try:
    os.mkdir(saving_path)
    print("saving_path directory created successfully")
except OSError as e:
    print("Error: %s" % (e.strerror))

data folder created successfully
Error: No such file or directory
Error: File exists


In [7]:
class LRScheduler():
    """
    Learning rate scheduler. If the validation loss does not decrease for the 
    given number of `patience` epochs, then the learning rate will decrease by
    by given `factor`.
    """
    def __init__(
        self, optimizer, patience=5, min_lr=1e-6, factor=0.5
    ):
        """
        new_lr = old_lr * factor
        :param optimizer: the optimizer we are using
        :param patience: how many epochs to wait before updating the lr
        :param min_lr: least lr value to reduce to while updating
        :param factor: factor by which the lr should be updated
        """
        self.optimizer = optimizer
        self.patience = patience
        self.min_lr = min_lr
        self.factor = factor
        self.lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( 
                self.optimizer,
                mode='min',
                patience=self.patience,
                factor=self.factor,
                min_lr=self.min_lr,
                verbose=True
            )
    def __call__(self, val_loss):
        self.lr_scheduler.step(val_loss)

In [8]:
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=10, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
    def __call__(self, score):
        if self.best_score == None:
            self.best_score = score
        elif self.best_score - score > self.min_delta:
            self.best_score = score
            self.counter = 0
        elif self.best_score - score < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [9]:
# Define transforms
train_transforms = transforms.Compose([#transforms.RandomRotation(30),
                                       #transforms.RandomResizedCrop(224),
                                       #transforms.RandomHorizontalFlip(),
                                       transforms.Resize(image_size),
                                       transforms.CenterCrop(image_size),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(image_size),
                                      transforms.CenterCrop(image_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [10]:
#@title
# Function - use some percent of data
def shorten_dataset(dataset, usage_percent=1.0):
  len_used = floor(len(dataset)*usage_percent)
  len_not_used = len(dataset) - len_used
  used_dataset, not_used_dataset = random_split(dataset, [len_used, len_not_used], generator=torch.Generator().manual_seed(manual_seed)) 
  return used_dataset

In [11]:
# Function - Split Data to Public & Private
def split_to_public_private(dataset, usage_percent=1.0, public_percent=0.25):
  len_used = floor(len(dataset)*usage_percent)
  len_not_used = len(dataset) - len_used
  used_dataset, not_used_dataset = random_split(dataset, [len_used, len_not_used], generator=torch.Generator().manual_seed(manual_seed)) 
  len_public = floor(len(used_dataset)*public_percent)
  len_private = len(used_dataset) - len_public
  return random_split(used_dataset, [len_public, len_private], generator=torch.Generator().manual_seed(manual_seed)) 
  

In [12]:
# Load Datas
train_set = datasets.CelebA(root='', download=False, split='train', target_type=["attr", "identity"], transform=train_transforms)
test_set = datasets.CelebA(root='', download=False, split='test', target_type=["attr", "identity"], transform=test_transforms)
valid_set = datasets.CelebA(root='', download=False, split='valid', target_type=["attr", "identity"], transform=test_transforms)

# shorten Dataset
if not use_whole_dataset:
  train_set = shorten_dataset(train_set, usage_percent)
  # test_set = shorten_dataset(test_set, usage_percent)
  valid_set = shorten_dataset(valid_set, usage_percent)

# DataLoader
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=workers, drop_last=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, num_workers=workers, drop_last=True)


In [13]:
# Decide which device we want to run on
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

In [14]:
# Plot some training images
# real_batch = next(iter(train_loader))
# plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.title("Training Images")
# plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [15]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    print(classname)
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [16]:
# Adversary Model - infer identity (just for usage_percent = 1.0) - or - infer sex (male/female) - or infer age (young/old)
# Age is selected. (young = 1)

# def AdvModel(pretrained=True, requires_grad=False):
#     model = models.densenet121(progress=True, pretrained=pretrained)
#     # either freeze or train the hidden layer parameters
#     if requires_grad == False:
#         for param in model.parameters():
#             param.requires_grad = False
#     elif requires_grad == True:
#         for param in model.parameters():
#             param.requires_grad = True
#     # make the classification layer learnable
#     adversaryClassifier = nn.Sequential(OrderedDict([
#                           ('fc1', nn.Linear(1024, 500)),
#                           ('relu', nn.ReLU()),
#                           ('fc2', nn.Linear(500, 2)),
#                           ('output', nn.LogSoftmax(dim=1))
#                           ]))

#     model.classifier = adversaryClassifier
#     return model

In [17]:
# Adversary Model - infer identity (just for usage_percent = 1.0) - or - infer sex (male/female) - or infer age (young/old)
# Age is selected. (young = 1)
class AdvModel(nn.Module):
    def __init__(self, ngpu):
        super(AdvModel, self).__init__()
        self.ngpu = ngpu
        
        # input is nc x 64 x 64 
        self.conv1 = nn.Conv2d(nc, nef, 4, 2, 1, bias=False)
        self.actv1 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 32 x 32
        self.conv2 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor2 = nn.BatchNorm2d(nef)
        self.actv2 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 16 x 16
        self.conv3 = nn.Conv2d(nef, nef, 4, 2, 1, bias=False)
        self.bnor3 = nn.BatchNorm2d(nef)
        self.actv3 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef) x 8 x 8
        self.conv4 = nn.Conv2d(nef, nef * 2, 4, 2, 1, bias=False)
        self.bnor4 = nn.BatchNorm2d(nef * 2)
        self.actv4 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (nef*2) x 4 x 4
        # shaping would be here: nef*2 x 4 x 4 -> 2048
        # state size. 2048
        self.fllc5 = nn.Linear(nef*2*4*4, nef*1*4*4)
        self.actv5 = nn.LeakyReLU(0.2, inplace=True)
        # state size. 1024

        # classifier: (age)
        self.fllc_features1 = nn.Linear(nef*1*4*4, nef*1*4*4)
        self.actv_features1 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_features1 = nn.Dropout(p=0.5)
        self.fllc_features2 = nn.Linear(nef*1*4*4, nef*4)
        self.actv_features2 = nn.LeakyReLU(0.2, inplace=True)
        self.dropout_features2 = nn.Dropout(p=0.5)
        self.fllc_features3 = nn.Linear(nef*4, nef)
        self.actv_features3 = nn.LeakyReLU(0.2, inplace=True)
        self.fllc_features4 = nn.Linear(nef, 2)
        self.actv_features4 = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Part 1:
        x = self.conv1(x)
        x = self.actv1(x)
        x = self.conv2(x)
        x = self.bnor2(x)
        x = self.actv2(x)
        x = self.conv3(x)
        x = self.bnor3(x)
        x = self.actv3(x)
        x = self.conv4(x)
        x = self.bnor4(x)
        x = self.actv4(x)
        # flatten
        x = torch.flatten(x, start_dim = 1)
        # Part 2:
        x = self.fllc5(x)
        x = self.actv5(x)
        # classifier: (age)
        y1 = self.fllc_features1(x)
        y1 = self.actv_features1(y1)
        y1 = self.dropout_features1(y1)
        y1 = self.fllc_features2(y1)
        y1 = self.actv_features2(y1)
        y1 = self.dropout_features2(y1)
        y1 = self.fllc_features3(y1)
        y1 = self.actv_features3(y1)
        y1 = self.fllc_features4(y1)
        y1 = self.actv_features4(y1)
        return y1


In [18]:
# Create the ADV
adversaryModel = AdvModel(ngpu).to(device)
# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    adversaryModel = nn.DataParallel(adversaryModel, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
adversaryModel.apply(weights_init)

Conv2d
LeakyReLU
Conv2d
BatchNorm2d
LeakyReLU
Conv2d
BatchNorm2d
LeakyReLU
Conv2d
BatchNorm2d
LeakyReLU
Linear
LeakyReLU
Linear
LeakyReLU
Dropout
Linear
LeakyReLU
Dropout
Linear
LeakyReLU
Linear
LogSoftmax
AdvModel


AdvModel(
  (conv1): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (actv1): LeakyReLU(negative_slope=0.2, inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (bnor2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (actv2): LeakyReLU(negative_slope=0.2, inplace=True)
  (conv3): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (bnor3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (actv3): LeakyReLU(negative_slope=0.2, inplace=True)
  (conv4): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (bnor4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (actv4): LeakyReLU(negative_slope=0.2, inplace=True)
  (fllc5): Linear(in_features=2048, out_features=1024, bias=True)
  (actv5): LeakyReLU(negative_slope=0.2, inplace=True)
  (fllc_features1):

In [19]:
# total parameters and trainable parameters
total_params = sum(p.numel() for p in adversaryModel.parameters())
print(f"{total_params:,} training parameters.")

3,692,482 training parameters.


In [20]:
adversaryCriterion = nn.NLLLoss() #nn.CrossEntropyLoss()
adversaryOptimizer = optim.Adam(adversaryModel.parameters(), lr=learning_rate, betas=(beta1, 0.999))

In [21]:
# Function - Save:
def save_model(name, number, model, res):
  checkpoint = {'res': res,
                'state_dict': model.state_dict()}
  torch.save(checkpoint, saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth')
  return True

In [22]:
# Function - Load:
def load_model(name, number, model, device):
  
  checkpoint = torch.load(saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth', map_location=device)
  res = checkpoint['res']
  model.load_state_dict(checkpoint['state_dict'])
  return {'model':model,
          'res':res}

In [23]:
# Function - Delete file:
def delete_file(name, number):
  address = saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth'
  try:
    os.remove(address)
    print("old file removed successfully")
  except OSError as e:
    print("Error: %s" % (e.strerror))

In [24]:
# Save Start Checkpoint

if(isFirstRun):
  adv_res = {'train_losses': [],
             'valid_losses': [],
             'test_y1_acc': [],
             'epoch_number': 0,
           };
  save_model('adv', 0, adversaryModel, adv_res)

In [25]:
# Load Last Checkpoint:
adv_load = load_model('adv', lastRunEpochNumber, adversaryModel, device)

train_losses = adv_load['res']['train_losses']
valid_losses = adv_load['res']['valid_losses']
test_y1_acc = adv_load['res']['test_y1_acc']
last_epoch = adv_load['res']['epoch_number']

In [26]:
# either early stopping or learning rate scheduler
if use_rate_schedule:
    print('INFO: Learning rate scheduler is on')
    lr_scheduler = LRScheduler(adversaryOptimizer)

if use_early_stop:
    print('INFO: Early stopping is on')
    early_stopping = EarlyStopping()


In [27]:
def fix_getting_data_problem(labels, decision_label):
    if use_whole_dataset:
        # return labels[0][:, 0, decision_label]
        return labels[0][:, decision_label]
    else:
        return labels[0][:, decision_label]

In [28]:
def extract_targets(labels):
    young_labels = fix_getting_data_problem(labels, using_index)
    old_labels = torch.add(1, -young_labels)
    young_target = torch.cat((young_labels.view(64,1), old_labels.view(64,1)), 1).float() # 64 is batch_size
    young_target = young_target.to(device)
    return young_target

In [29]:
# Function - training function
def fit(model, train_loader, optimizer, criterion):
    print('Training')
    model.train()

    train_loss = 0.0
    prog_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, data in prog_bar:
        inputs, labels = data[0], data[1]
        inputs = inputs.to(device)
        # young_target = extract_targets(labels)
        young_target = labels[0][:, using_index]
        young_target = young_target.to(device)
        model.zero_grad()
        outputs = model.forward(inputs)
        loss = criterion(outputs, young_target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()          
    train_loss = train_loss / len(train_loader)
    return train_loss

In [30]:
# Function - validation function
def validate(model, valid_loader, criterion):
    print('\nValidating')
    model.eval()
    valid_loss = 0.0

    prog_bar = tqdm(enumerate(valid_loader), total=len(valid_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[1]
            inputs = inputs.to(device)
            # young_target = extract_targets(labels)
            young_target = labels[0][:, using_index]
            young_target = young_target.to(device)
            outputs = model.forward(inputs)
            loss = criterion(outputs, young_target)
            valid_loss += loss.item()
        valid_loss = valid_loss / len(valid_loader)
        return valid_loss

In [31]:
# Calc Accuracy
def calcAccuracyTest(model, test_loader):
    model.to(device)
    print("\nCalculating Accuracy...")
    model.eval()
    y1_accuracy = 0
    prog_bar = tqdm(enumerate(test_loader), total=len(test_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[1]
            inputs = inputs.to(device)
            # young_target = extract_targets(labels)
            young_target = labels[0][:, using_index]
            young_target = young_target.to(device)
            # a1, young_label_top_class = young_target.topk(1, dim=1)
            output = model(inputs)
            ps_y1 = torch.exp(output)
            top_p_y1, top_class_y1 = ps_y1.topk(1, dim=1)
            # equals_y1 = top_class_y1 == young_label_top_class.view(*top_class_y1.shape)
            equals_y1 = top_class_y1 == young_target.view(*top_class_y1.shape)
            acc_y1 = equals_y1.sum().item()
            y1_accuracy += (acc_y1 / len(equals_y1))            
    y1_accuracy = y1_accuracy / len(test_loader)
    return y1_accuracy

In [ ]:






# Training Loop
adversaryModel.to(device)
calc_every_epoch = 1
save_every_epoch = 1

start = time.time()
print("Starting Training Loop...")

for epoch in range(last_epoch+1, num_epochs+1):
    print(f"Epoch {epoch}/{num_epochs}: ")
    train_loss = fit(adversaryModel, train_loader, adversaryOptimizer, adversaryCriterion)
    valid_loss = validate(adversaryModel, valid_loader, adversaryCriterion)
    y1_accuracy = calcAccuracyTest(adversaryModel, test_loader)
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    test_y1_acc.append(y1_accuracy)

    adv_res = {'train_losses': train_losses,
               'valid_losses': valid_losses,
               'test_y1_acc': test_y1_acc,
               'epoch_number': epoch
                }
    save_model('adv', 'last', adversaryModel, adv_res)
    if epoch % save_every_epoch == 0:
      # if epoch-save_every_epoch>=0:
      #   delete_file('adv', epoch-save_every_epoch)
        save_model('adv', epoch, adversaryModel, adv_res)

    if use_rate_schedule:
        lr_scheduler(valid_loss)
    if use_early_stop:
        early_stopping(valid_loss)
        if early_stopping.early_stop:
            break

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Valid Loss: {valid_loss:.4f}")
    print(f"Young Accuracy on Testset: {y1_accuracy:.4f}")

end = time.time()
print(f"Training time: {(end-start)/60:.3f} minutes")

print('TRAINING COMPLETE')

Starting Training Loop...
Epoch 91/200: 
Training


100%|██████████| 2543/2543 [04:03<00:00, 10.44it/s]


Validating



100%|██████████| 310/310 [00:25<00:00, 12.18it/s]


Calculating Accuracy...


...


100%|██████████| 310/310 [00:26<00:00, 11.69it/s]


Calculating Accuracy...



100%|██████████| 311/311 [00:24<00:00, 12.77it/s]


Train Loss: 0.0276
Valid Loss: 0.7130
Young Accuracy on Testset: 0.9714
Epoch 134/200: 
Training


  7%|▋         | 172/2543 [00:15<05:37,  7.02it/s]

In [ ]:
# # modified dataset:
# ref_project = 'utl-trn-2'

# modified_dataset_saving_path = 'drive/MyDrive/ML/modifiedDatasets/' + ref_project + '/'
# #'C:/Users/jamshidi/modifiedDatasets/' + ref_project + '/'
# #'C:/Users/Mohammad/modifiedDatasets/' + ref_project + '/'
# # 'drive/MyDrive/ML/modifiedDatasets/' + ref_project + '/'

# # load modified dataset:
# m_transforms = transforms.Compose([transforms.ToTensor()])
# m_test_set = datasets.CelebA(root=modified_dataset_saving_path, download=False, split='test', target_type=["attr", "identity"], transform=m_transforms)
# m_test_loader = torch.utils.data.DataLoader(m_test_set, batch_size=batch_size, num_workers=workers, drop_last=True)

# # Test on obfuscated data
# adversaryModel.to(device)
# young_accuracy = calcAccuracyTest(adversaryModel, m_test_loader)
# print(f"Young Accuracy on Testset: {young_accuracy:.2f}")

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print('Loss plot...')

# loss plots
plt.figure(figsize=(10,7))
plt.title("Train-Valid Loss Trend")
plt.plot(train_losses, color='green', label='Training Loss')
plt.plot(valid_losses, color='blue', label='Validation Loss')
plt.legend(frameon=False)
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.ylim([0,0.02])
plt.savefig(saving_path + "loss_plot.png")
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.title("smiling Accuracy Trend")
plt.plot(test_y1_acc, color='green', label='smiling Test set Accuracy')
plt.legend(frameon=False)
plt.xlabel("epochs")
plt.ylabel("Accuracy")
plt.savefig(saving_path + "accuracy_test_plot.png")
plt.show()

In [ ]:
import operator
indexed = list(enumerate(test_y1_acc))
top_5 = sorted(indexed, key=operator.itemgetter(1))[-5:]
print(list(reversed([i for i, v in top_5])))
print(list(reversed([v for i, v in top_5])))

In [ ]:
import operator
indexed = list(enumerate(train_losses))
top_5 = sorted(indexed, key=operator.itemgetter(1))[:5]
print(list([i for i, v in top_5]))
print(list([v for i, v in top_5]))

In [ ]:
import operator
indexed = list(enumerate(valid_losses))
top_5 = sorted(indexed, key=operator.itemgetter(1))[:5]
print(list([i for i, v in top_5]))
print(list([v for i, v in top_5]))